In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import re
import requests
import bs4

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

import numpy as np

In [ ]:
# Set up selenium to use Firefox
options = Options()
options.headless = True #No need to open a browser window

# Example of manaully specifying the WebDriver's location: 
driver = webdriver.Firefox(executable_path="../Others/geckodriver.exe",options=options)

In [ ]:
def racecard(url, date, match):
    driver.get(url)
    
    # Is there anything?
    if driver.page_source.find("No information.") != -1:
        return []
    
    # Wait 10 secs so that the dynamic content has time to load.
    # Proceed to next date if page doesn't load.
    try:
        wait = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(((By.XPATH, 
                                             '//table[@class="starter f_tac f_fs13 draggable hiddenable"]'))))
    except Exception as e: 
        print(e, date, match)
        return []
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    table = soup.find('table', class_ = 'starter f_tac f_fs13 draggable hiddenable')

    output_list = [] 
    
    for tr in table.find('tbody').find_all('tr'):
        cols = []
        racecard = [td.string for td in tr('td')]

        cols.extend(racecard)
        
        cols.append(date)
        cols.append(match)
        
        output_list.append(cols)
 
    return output_list

In [ ]:
filepath = "racecard_git.csv"

from datetime import date
import time

today = date.today().strftime("%Y%m%d")
url_front = 'https://racing.hkjc.com/racing/information/English/Racing/RaceCard.aspx?RaceDate='


import csv
with open(filepath, 'w', newline='') as csvfile:
    mywriter = csv.writer(csvfile)
    mywriter.writerow(['Horse No.', 'Last 6 Runs', 'Colour', 'Horse', 'Brand No.', 'Wt.', 'Jockey', 'Over Wt.', 
                       'Draw', 'Trainer', "Int'l Rtg.", 'Rtg.', 'Rtg.+/-', 'Horse Wt. (Declaration)', 'Wt.+/- (vs Declaration)', 'Best Time', 'Age',
                       'WFA', 'Sex', 'Season Stakes', 'Priority', 'Gear', 'Owner', 'Sire', 'Dam', 'Import Cat.', "date", "match"]
                     )
    
    #Copy the loop from above and incorporate the csv-saving code
    for match in range(1, 13):
        start_time = time.time()
        url = url_front + str(today) + '&RaceNo='+ str(match)
        
        content = racecard(url, today, match)
        content = np.array(content)
        if len(content) > 0:
            mywriter.writerows(content)
            print(today, match, "saved.")